<a href="https://colab.research.google.com/github/IndigoClay/cudaCompileDeviceQuery/blob/main/cudaCompileDeviceQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Thu Mar 31 10:33:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |            

In [17]:
!git clone https://github.com/IndigoClay/cudaCompileDeviceQuery.git

Cloning into 'cudaCompileDeviceQuery'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 19 (delta 5), reused 17 (delta 3), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [8]:
!pip install git+https://github.com/IndigoClay/nvcc4jupyter.git

  Cloning https://github.com/IndigoClay/nvcc4jupyter.git to /tmp/pip-req-build-r07ux7rl
  Running command git clone -q https://github.com/IndigoClay/nvcc4jupyter.git /tmp/pip-req-build-r07ux7rl
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4286 sha256=fc0c63d4b0bf2199d8c840ed02b7b6d10775bcf806f45c79e92f4963948c1745
  Stored in directory: /tmp/pip-ephem-wheel-cache-96tfapod/wheels/c4/52/63/f7029f491c28a55cf4cb96d4abbcb3460592100edbd8560c5e
Successfully built NVCCPlugin


In [9]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [10]:
%%cu
#include <stdio.h>

#define NUM_BLOCKS 16
#define BLOCK_WIDTH 1

__global__ void hello()
{
    printf("Hello world! I'm a thread in block %d\n", blockIdx.x);
}


int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_WIDTH>>>();

    // force the printf()s to flush
    cudaDeviceSynchronize();
    
    printf("That's all!\n");

    return 0;
}

Hello world! I'm a thread in block 12
Hello world! I'm a thread in block 9
Hello world! I'm a thread in block 8
Hello world! I'm a thread in block 7
Hello world! I'm a thread in block 6
Hello world! I'm a thread in block 10
Hello world! I'm a thread in block 4
Hello world! I'm a thread in block 3
Hello world! I'm a thread in block 11
Hello world! I'm a thread in block 5
Hello world! I'm a thread in block 1
Hello world! I'm a thread in block 0
Hello world! I'm a thread in block 2
Hello world! I'm a thread in block 14
Hello world! I'm a thread in block 13
Hello world! I'm a thread in block 15
That's all!



In [18]:
%%cu
#include <cuda_runtime.h>
#include "/content/cudaCompileDeviceQuery/helper_cuda.h"
#include <iostream>
using namespace std;
__global__ void use_local_memory_GPU(float in)
{
    float f;
    f = in;
}

int main(int argc, char **argv)
{
    use_local_memory_GPU<<<1, 128>>>(2.0f);
    //cout << "hello there";
    //system("ls");
}

In [27]:
!cd /content/cudaCompileDeviceQuery && pwd && make

/content/cudaCompileDeviceQuery
/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common  -m64    --std=c++11 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_86,code=compute_86 -o deviceQuery.o -c deviceQuery.cpp
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
/usr/local/cuda/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=

In [28]:
!./cudaCompileDeviceQuery/deviceQuery

./cudaCompileDeviceQuery/deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla K80"
  CUDA Driver Version / Runtime Version          11.2 / 11.1
  CUDA Capability Major/Minor version number:    3.7
  Total amount of global memory:                 11441 MBytes (11996954624 bytes)
  (013) Multiprocessors, (192) CUDA Cores/MP:    2496 CUDA Cores
  GPU Max Clock rate:                            824 MHz (0.82 GHz)
  Memory Clock rate:                             2505 Mhz
  Memory Bus Width:                              384-bit
  L2 Cache Size:                                 1572864 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(65536), 2D=(65536, 65536), 3D=(4096, 4096, 4096)
  Maximum Layered 1D Texture Size, (num) layers  1D=(16384), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(16384, 16384), 2048 layers
  Total amount of constant memory:               65536 bytes
  To

In [29]:
!pwd

/content
